<a href="https://colab.research.google.com/github/Bobisreallyme/TestRepo/blob/main/Cleaned_PretrainingDistilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import sqlite3
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import AutoModelForMaskedLM, AutoTokenizer
from datasets import concatenate_datasets
from transformers import DataCollatorForLanguageModeling
from tqdm.auto import tqdm
import torch
import math

In [ ]:
streams_of_interest_file = r'YourStreamsOfInterest.ods'
database_filename = r'YourDatabase.db'

In [ ]:
#DEFINE STREAMS OF INTEREST
Alldataframe = pd.read_excel(streams_of_interest_file)

ListofStreamUrls = Alldataframe.iloc[:,2].tolist()
ListofStreamNames = Alldataframe.iloc[:,1].tolist()

streams_of_interest = []
for i in range(93,300+93,2):
    streams_of_interest.append(ListofStreamUrls[i])


#CONNECT TO DATABASE OF INTEREST
all_messages = []
all_times = []
all_names = []
donor_names = []
ChatfileCM = sqlite3.connect(database_filename)
CursorCM = ChatfileCM.cursor()

#GET DATA OFF DATABASE. REMOVE ALL STREAMS WHERE THERE IS NO DATA
for index,url_temp in enumerate(streams_of_interest):

    all_messages_temp = []
    all_times_temp = []
    all_names_temp = []

    Messagetype = 'text_message'
    CursorCM.execute("SELECT ListofMessages FROM DBdb WHERE URL = ? ORDER BY id", (url_temp,))
    StreamoneText0 = CursorCM.fetchall()
    Testmessages = []
    for i in range(0,len(StreamoneText0)):
        Testmessages.append(StreamoneText0[i][0])
    all_messages_temp.append(Testmessages)


    CursorCM.execute("SELECT ListofTimes FROM DBdb WHERE URL = ? ORDER BY id", (url_temp,))
    StreamoneText0 = CursorCM.fetchall()
    Testtimes = []
    for i in range(0,len(StreamoneText0)):
        Testtimes.append(StreamoneText0[i][0])
    all_times_temp.append(Testtimes)


    CursorCM.execute("SELECT ListofAuthors FROM DBdb WHERE URL = ? ORDER BY id", (url_temp,))
    StreamoneText0 = CursorCM.fetchall()
    Testnames = []
    for i in range(0,len(StreamoneText0)):
        Testnames.append(StreamoneText0[i][0])
    all_names_temp.append(Testnames)

    if len(all_messages_temp[0]) > 0:
        all_messages.append(all_messages_temp[0])
        all_times.append(all_times_temp[0])
        all_names.append(all_names_temp[0])
    else:
        print(index)


#FUNCTION TO SORT MESSAGES AND SUCH BY TIME
def message_sorter(times,messages,names):
    zipped = zip(np.array(times).astype(float),messages,names)
    sorted_zipped = sorted(zipped, key=lambda x: x[0])
    sorted_times, sorted_messages,sorted_names = zip(*sorted_zipped)
    return sorted_times,sorted_messages,sorted_names

#SORT DATA
sorted_times = []
sorted_messages = []
sorted_names = []

for index,temp_mess in enumerate(all_messages):
    temp_sorted_times,temp_sorted_messages,temp_sorted_names = message_sorter(all_times[index],all_messages[index],all_names[index])
    sorted_times.append(list(temp_sorted_times))
    sorted_messages.append(list(temp_sorted_messages))
    sorted_names.append(list(temp_sorted_names))


#REMOVE A RANDOM SET OF DATA FOR PERPLEXITY CALCS LATER
#TOKENIZE FUNCTION
def tokenize_function(examples):
    result = tokenizer(examples["messages"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

#LOAD MODEL FROM HUGGINGFACE
model_checkpoint = "distilroberta-base"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


lm_datasets_aggregate = []
test_dataset_aggregate = []
chunk_size = 128

for index,temp_sorted_mess in enumerate(sorted_messages):

    #REMOVE ALL NONES
    sorted_messages_processed = []
    for index,message in enumerate(temp_sorted_mess):
        if message!=None:
            sorted_messages_processed.append(message)


    #SPLIT THE DATASET INTO A SECTION FOR PERPLEXITY ON ORGINAL DATASET FORMAT AND THE MAIN DATASET (THIS IS A SECOND TEST DATASET TO MAKE SURE THAT CONCATENATION DOESN'T RESULT IN LOSS OF PREDICTION ABILITY ON ORIGINAL DATASET)
    length_removed = int(len(sorted_messages_processed)*0.0005)
    indices_to_remove = np.random.choice(len(sorted_messages_processed), length_removed, replace=False)
    sorted_message_main = [message_temp for index,message_temp in enumerate(sorted_messages_processed) if index not in indices_to_remove]
    test_dataset = [message_temp for index,message_temp in enumerate(sorted_messages_processed) if index in indices_to_remove]

    test_dataset_aggregate = test_dataset_aggregate + test_dataset

    temp_dict = {"messages":sorted_message_main}
    temp_dataset = Dataset.from_dict(temp_dict)

    #BATCH TOKENIZE
    tokenized_datasets = temp_dataset.map(
        tokenize_function, batched=True, remove_columns=["messages"]
    )

    def group_texts(examples):
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        #DROP LAST CHUNK IF IT'S SMALLER THAN CHUNK SIZE
        total_length = (total_length // chunk_size) * chunk_size
        result = {
            k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result

    lm_datasets = tokenized_datasets.map(group_texts, batched=True)
    lm_datasets_aggregate.append(lm_datasets)


#PULL ALL THE DATASETS TOGETHER INTO ONE DATASET OBJECT
lm_datasets_concatenated = lm_datasets_aggregate[0]
for index in range(1,len(lm_datasets_aggregate)):
    lm_datasets_concatenated = concatenate_datasets([lm_datasets_concatenated,lm_datasets_aggregate[index]])

#MASK THE DATA FOR TRAINING AND TESTING
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

#SPLIT INTO TRAIN AND TEST DATASETS
train_size = int(total_number_samples*0.9)
test_size = int(total_number_samples*0.1)

downsampled_dataset = lm_datasets_concatenated.train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset


downsampled_dataset = downsampled_dataset.remove_columns(["word_ids"])
eval_dataset = downsampled_dataset["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=downsampled_dataset["test"].column_names,
)


#PROCESS THE ORIGINAL LENGTH DATASET
def tokenize_function_2(examples):
    return tokenizer(examples["messages"], padding="max_length", truncation=True)

test_dict = {"messages":test_dataset_aggregate}
test_dataset_dict = Dataset.from_dict(test_dict)

tokenized_test_dataset = test_dataset_dict.map(
    tokenize_function_2, batched=True, remove_columns=["messages"]
)


eval_dataset_2 = tokenized_test_dataset.map(
    insert_random_mask,
    batched=True,
    remove_columns=['input_ids', 'attention_mask'],
)


eval_dataset_2 = eval_dataset_2.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)


eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)


from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 16
train_dataloader = DataLoader(
    downsampled_dataset["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)


eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)


eval_dataloader_2 = DataLoader(
    eval_dataset_2, batch_size=batch_size, collate_fn=default_data_collator
)



#SEND MODEL TO THE GPU
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

#SET UP SAVEPATH
default_path_tosave = r"C:\Users\BobsBrain\Contrast_Learning\Model_from_concatentation_4_26_2024_"


#INITIALIZE TRAINING PARAMETERS
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

from accelerate import Accelerator
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

from transformers import get_scheduler
num_train_epochs = 10
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)



#TRAIN AND EVALUATE MODEL

model.eval()
#CALCULATE THE INITIAL MODEL PERPLEXITY ON CONCATENATED DATA
losses = []
for step, batch in enumerate(eval_dataloader):
    with torch.no_grad():
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

    loss = outputs.loss
    losses.append(accelerator.gather(loss.repeat(batch_size)))

losses = torch.cat(losses)
losses = losses[: len(eval_dataset)]
try:
    perplexity = math.exp(torch.mean(losses))
except OverflowError:
    perplexity = float("inf")

print(f"Perplexity on Concat: {perplexity}")

#CALCULATE THE INITIAL MODEL PERPLEXITY ON NON CONCATENATED DATA
losses = []
for step, batch in enumerate(eval_dataloader_2):
    with torch.no_grad():
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

    loss = outputs.loss
    losses.append(accelerator.gather(loss.repeat(batch_size)))

losses = torch.cat(losses)
losses = losses[: len(eval_dataset_2)]
try:
    perplexity = math.exp(torch.mean(losses))
except OverflowError:
    perplexity = float("inf")

print(f"Perplexity: {perplexity}")

#SAVE INITIAL MODEL


#MODEL TRAINING LOOP
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)


    #SAVE THE MODEL AFTER TRAINING
    model.save_pretrained(default_path_tosave + str(epoch+11))

    #EVALUATION ON CONCAT DATASET
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity on Concat: {perplexity}")



    #EVALUATION ON NON CONCAT DATASET
    losses = []
    for step, batch in enumerate(eval_dataloader_2):
        with torch.no_grad():
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset_2)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

